In [3]:
import pickle as pkl
import pandas as pd
with open("SelectedJobData_0801-1001.pkl", "rb") as f:
    object = pkl.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'data.csv')

In [9]:
import pandas as pd

fields = ['_id', 'group', 'owner', 'job_name', 'job_number', 'slots', 'mem', 'submission_time', 'start_time', 'end_time']

df = pd.read_csv('data/raw_data.csv', skipinitialspace=True, usecols=fields)

df.to_csv(r'data/data.csv', index = False)
